# 在 Azure Kubernetes Service (AKS) 上使用 LLaMA-Factory Docker 部署微调模型

本教程将指导您使用 LLaMA-Factory Docker 镜像在 AKS 上部署微调后的模型，实现一个可扩展的模型推理服务。

## 目录
1. [前置条件](#前置条件)
2. [环境准备](#环境准备)
3. [创建 AKS 集群](#创建-aks-集群)
4. [配置 Container Registry](#配置-container-registry)
5. [创建 LLaMA-Factory 部署文件](#创建-llamafactory-部署文件)
6. [部署到 AKS](#部署到-aks)
7. [配置负载均衡器](#配置负载均衡器)
8. [测试模型端点](#测试模型端点)
9. [监控和扩展](#监控和扩展)
10. [清理资源](#清理资源)

## 前置条件

### 系统要求
- Azure 订阅（如果没有，请创建[免费账户](https://azure.microsoft.com/free/)）
- Azure CLI 2.50.0 或更高版本
- kubectl 命令行工具
- Docker 命令行工具
- 至少 16GB RAM 的开发机器

### 安装必要工具

首先，我们需要安装和配置必要的工具。以下命令将安装 Azure CLI、kubectl 和相关扩展：

In [ ]:
# 安装 Azure CLI (如果尚未安装)
!curl -sL https://aka.ms/InstallAzureCLIDeb | sudo bash

# 验证安装
!az --version

# 安装 kubectl
!az aks install-cli

# 安装 k8s-extension 扩展
!az extension add --name k8s-extension

# 更新扩展到最新版本
!az extension update --name k8s-extension

### 登录 Azure

使用以下命令登录到 Azure 并设置默认订阅：

In [ ]:
import os
import random

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "llama-aks-rg"
os.environ["LOCATION"] = "eastus"
os.environ["AKS_CLUSTER_NAME"] = "llama-aks-cluster"
os.environ["ACR_NAME"] = f"llamaacr{random.randint(0, 999999)}"
os.environ["NAMESPACE"] = "llama-inference"
os.environ["SERVICE_NAME"] = "llama-service"
os.environ["DEPLOYMENT_NAME"] = "llama-deployment"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"AKS Cluster: {os.environ['AKS_CLUSTER_NAME']}")
print(f"ACR Name: {os.environ['ACR_NAME']}")
print(f"Namespace: {os.environ['NAMESPACE']}")

In [ ]:
import os
import random

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "phi4-aks-rg"
os.environ["LOCATION"] = "eastus"
os.environ["AKS_CLUSTER_NAME"] = "phi4-aks-cluster"
os.environ["ML_WORKSPACE_NAME"] = "phi4-ml-workspace"
os.environ["EXTENSION_NAME"] = "phi4-ml-extension"
os.environ["COMPUTE_NAME"] = "phi4-k8s-compute"
os.environ["ENDPOINT_NAME"] = "phi4-endpoint"
os.environ["DEPLOYMENT_NAME"] = "phi4-deployment"
os.environ["ACR_NAME"] = f"phi4acr{random.randint(0, 999999)}"
os.environ["STORAGE_ACCOUNT"] = f"phi4storage{random.randint(0, 999999)}"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"AKS Cluster: {os.environ['AKS_CLUSTER_NAME']}")
print(f"ML Workspace: {os.environ['ML_WORKSPACE_NAME']}")

## 创建 AKS 集群

### 1. 创建资源组

首先，我们需要创建一个资源组来管理所有相关资源：

In [ ]:
# 创建资源组
!az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

# 验证资源组创建
!az group show --name $RESOURCE_GROUP

### 2. 创建 AKS 集群

创建一个适合运行 LLaMA 模型的 AKS 集群：

In [ ]:
!az aks create \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME \
    --node-count 2 \
    --node-vm-size Standard_NC6s_v3 \
    --enable-managed-identity \
    --generate-ssh-keys \
    --enable-addons monitoring \
    --enable-cluster-autoscaler \
    --min-count 1 \
    --max-count 3 \
    --node-osdisk-size 100

# 获取 AKS 凭据
!az aks get-credentials \
    --resource-group $RESOURCE_GROUP \
    --name $AKS_CLUSTER_NAME \
    --overwrite-existing

# 验证集群连接
!kubectl get nodes

### 3. 安装 GPU 驱动程序（如果使用 GPU 节点）

为 AKS 集群配置 GPU 支持：

In [ ]:
# 安装 NVIDIA GPU 设备插件
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/v0.14.1/nvidia-device-plugin.yml

# 验证 GPU 节点
!kubectl get nodes -o wide

# 检查 GPU 资源
!kubectl describe nodes | grep -A 5 "Capacity\|Allocatable" | grep nvidia

## 配置 Container Registry

### 1. 创建 Azure Container Registry

In [ ]:
# 创建 Azure Container Registry
!az acr create \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --location $LOCATION \
    --sku Standard \
    --admin-enabled true

登录到ACR

In [ ]:
!az acr login --name $ACR_NAME

In [ ]:

%%bash
# 获取 ACR 登录服务器
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)
echo "ACR Login Server: $ACR_LOGIN_SERVER"

# 将 ACR 与 AKS 集群关联
!az aks update \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --attach-acr $ACR_NAME

### 2. 准备 LLaMA-Factory Docker 镜像

我们可以直接使用官方的 LLaMA-Factory 镜像，或者自定义镜像：

In [ ]:
%%bash
# 获取 ACR 登录服务器地址
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)

In [ ]:
# 拉取官方 LLaMA-Factory 镜像
!docker pull registry.cn-hangzhou.aliyuncs.com/modelscope-repo/llamafactory:latest

# 标记镜像
!docker tag registry.cn-hangzhou.aliyuncs.com/modelscope-repo/llamafactory:latest ${ACR_LOGIN_SERVER}/llamafactory:latest

# 推送到 ACR
!docker push ${ACR_LOGIN_SERVER}/llamafactory:latest

# 验证镜像推送
!az acr repository list --name $ACR_NAME

## 创建 LLaMA-Factory 部署文件

### 1. 创建 Namespace 和 ConfigMap

In [ ]:
# 创建命名空间
!kubectl create namespace $NAMESPACE

# 创建配置文件目录
!mkdir -p k8s-manifests

# 验证命名空间创建
!kubectl get namespaces

In [ ]:
%%writefile k8s-manifests/configmap.yaml
apiVersion: v1
kind: ConfigMap
metadata:
  name: llama-config
  namespace: llama-inference
data:
  API_HOST: "0.0.0.0"
  API_PORT: "8000"
  WEBUI_HOST: "0.0.0.0" 
  WEBUI_PORT: "7860"
  MODEL_NAME: "your-model-name"  # 替换为您的模型名称
  ADAPTER_NAME: "your-adapter-path"  # 如果有微调适配器的话
  QUANTIZATION_BIT: "4"
  TEMPLATE: "default"
  HF_ENDPOINT: "https://hf-mirror.com"

In [ ]:
%%writefile k8s-manifests/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: phi4-deployment
  namespace: phi4-inference
  labels:
    app: phi4
spec:
  replicas: 1
  selector:
    matchLabels:
      app: phi4
  template:
    metadata:
      labels:
        app: phi4
    spec:
      containers:
      - name: llamafactory
        image: ${ACR_LOGIN_SERVER}/llamafactory:latest
        ports:
        - containerPort: 8000
          name: api
        - containerPort: 7860
          name: webui
        env:
        - name: CUDA_VISIBLE_DEVICES
          value: "0"
        - name: HF_HOME
          value: "/app/models"
        - name: HF_HUB_CACHE
          value: "/app/models"
        envFrom:
        - configMapRef:
            name: phi4-config
        command: ["/bin/bash"]
        args:
        - -c
        - |
          # 启动 API 服务
          llamafactory-cli api \
            --model_name_or_path $(MODEL_NAME) \
            --template $(TEMPLATE) \
            --quantization_bit $(QUANTIZATION_BIT) \
            --host $(API_HOST) \
            --port $(API_PORT) &
          
          # 启动 Web UI
          llamafactory-cli webui \
            --host $(WEBUI_HOST) \
            --port $(WEBUI_PORT) &
          
          # 保持容器运行
          wait
        resources:
          requests:
            memory: "8Gi"
            cpu: "2"
            nvidia.com/gpu: 1
          limits:
            memory: "16Gi"
            cpu: "4"
            nvidia.com/gpu: 1
        volumeMounts:
        - name: model-cache
          mountPath: /app/models
        - name: shm
          mountPath: /dev/shm
        readinessProbe:
          httpGet:
            path: /docs
            port: 8000
          initialDelaySeconds: 300
          periodSeconds: 30
          timeoutSeconds: 10
        livenessProbe:
          httpGet:
            path: /docs
            port: 8000
          initialDelaySeconds: 600
          periodSeconds: 60
          timeoutSeconds: 30
      volumes:
      - name: model-cache
        emptyDir:
          sizeLimit: 50Gi
      - name: shm
        emptyDir:
          medium: Memory
          sizeLimit: 8Gi
      tolerations:
      - key: "sku"
        operator: "Equal"
        value: "gpu"
        effect: "NoSchedule"
      nodeSelector:
        accelerator: nvidia

In [ ]:
%%writefile k8s-manifests/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: phi4-service
  namespace: phi4-inference
  labels:
    app: phi4
spec:
  type: LoadBalancer
  ports:
  - name: api
    port: 8000
    targetPort: 8000
    protocol: TCP
  - name: webui
    port: 7860
    targetPort: 7860
    protocol: TCP
  selector:
    app: phi4
---
apiVersion: v1
kind: Service
metadata:
  name: phi4-api-service
  namespace: phi4-inference  
  labels:
    app: phi4
spec:
  type: ClusterIP
  ports:
  - name: api
    port: 8000
    targetPort: 8000
    protocol: TCP
  selector:
    app: phi4
---
apiVersion: v1
kind: Service
metadata:
  name: phi4-webui-service
  namespace: phi4-inference
  labels:
    app: phi4
spec:
  type: ClusterIP
  ports:
  - name: webui
    port: 7860
    targetPort: 7860
    protocol: TCP
  selector:
    app: phi4

In [ ]:
%%writefile k8s-manifests/service.yaml
apiVersion: v1
kind: Service
metadata:
  name: llama-service
  namespace: llama-inference
  labels:
    app: llama
spec:
  type: LoadBalancer
  ports:
  - name: api
    port: 8000
    targetPort: 8000
    protocol: TCP
  - name: webui
    port: 7860
    targetPort: 7860
    protocol: TCP
  selector:
    app: llama
---
apiVersion: v1
kind: Service
metadata:
  name: llama-api-service
  namespace: llama-inference  
  labels:
    app: llama
spec:
  type: ClusterIP
  ports:
  - name: api
    port: 8000
    targetPort: 8000
    protocol: TCP
  selector:
    app: llama
---
apiVersion: v1
kind: Service
metadata:
  name: llama-webui-service
  namespace: llama-inference
  labels:
    app: llama
spec:
  type: ClusterIP
  ports:
  - name: webui
    port: 7860
    targetPort: 7860
    protocol: TCP
  selector:
    app: llama

In [ ]:
%%bash
# 获取 ACR 登录服务器地址
ACR_LOGIN_SERVER=$(az acr show --name $ACR_NAME --resource-group $RESOURCE_GROUP --query loginServer -o tsv)

# 替换部署文件中的镜像地址
!sed -i "s/\${ACR_LOGIN_SERVER}/$ACR_LOGIN_SERVER/g" k8s-manifests/deployment.yaml

# 应用所有配置文件
!kubectl apply -f k8s-manifests/

# 检查部署状态
!kubectl get pods -n $NAMESPACE
!kubectl get services -n $NAMESPACE

In [ ]:
# 等待 Pod 就绪
!kubectl wait --for=condition=ready pod -l app=phi4 -n $NAMESPACE --timeout=600s

# 查看详细状态
!kubectl describe pods -l app=phi4 -n $NAMESPACE

# 查看日志
!kubectl logs -l app=phi4 -n $NAMESPACE --tail=50

# 检查服务状态
!kubectl get svc -n $NAMESPACE -o wide

In [ ]:
# 等待外部 IP 分配
!kubectl get service phi4-service -n $NAMESPACE --watch

# 获取外部 IP（这可能需要几分钟时间）
import time
import subprocess

def get_external_ip():
    for _ in range(60):  # 等待最多10分钟
        result = subprocess.run([
            'kubectl', 'get', 'service', 'phi4-service', 
            '-n', os.environ['NAMESPACE'],
            '-o', 'jsonpath={.status.loadBalancer.ingress[0].ip}'
        ], capture_output=True, text=True)
        
        if result.stdout and result.stdout.strip():
            return result.stdout.strip()
        
        print("等待外部 IP 分配...")
        time.sleep(10)
    
    return None

external_ip = get_external_ip()
if external_ip:
    print(f"外部 IP 地址: {external_ip}")
    print(f"API 端点: http://{external_ip}:8000")
    print(f"Web UI: http://{external_ip}:7860")
else:
    print("未能获取外部 IP 地址，请手动检查")

In [ ]:
# 测试 API 健康状态
!curl -X GET http://{external_ip}:8000/v1/models

# 测试文档页面
!curl -X GET http://{external_ip}:8000/docs

### 2. 使用 Python 测试 API

In [ ]:
import requests
import json

# API 端点
api_url = f"http://{external_ip}:8000/v1/chat/completions"

# 测试数据
test_cases = [
    {
        "model": "microsoft/Phi-4",
        "messages": [
            {"role": "user", "content": "解释什么是量子计算？"}
        ],
        "max_tokens": 200,
        "temperature": 0.7
    },
    {
        "model": "microsoft/Phi-4", 
        "messages": [
            {"role": "user", "content": "写一个 Python 函数计算斐波那契数列"}
        ],
        "max_tokens": 300,
        "temperature": 0.5
    }
]

# 发送测试请求
for i, test_data in enumerate(test_cases, 1):
    print(f"\n=== 测试 {i} ===")
    print(f"问题: {test_data['messages'][0]['content']}")
    print("-" * 50)
    
    try:
        response = requests.post(
            api_url,
            headers={"Content-Type": "application/json"},
            json=test_data,
            timeout=120
        )
        
        if response.status_code == 200:
            result = response.json()
            content = result.get('choices', [{}])[0].get('message', {}).get('content', '')
            usage = result.get('usage', {})
            
            print(f"回答: {content}")
            print(f"令牌使用: {usage}")
        else:
            print(f"请求失败: {response.status_code}")
            print(f"错误信息: {response.text}")
            
    except requests.exceptions.RequestException as e:
        print(f"请求异常: {e}")
    except Exception as e:
        print(f"其他错误: {e}")

In [ ]:
# 查看 Pod 资源使用情况
!kubectl top pods -n $NAMESPACE

# 查看节点资源使用情况  
!kubectl top nodes

# 查看 Pod 事件
!kubectl get events -n $NAMESPACE --sort-by='.lastTimestamp'

# 实时监控日志
!kubectl logs -f -l app=phi4 -n $NAMESPACE

In [ ]:
# 手动扩展 Deployment
!kubectl scale deployment phi4-deployment --replicas=2 -n $NAMESPACE

# 查看扩展状态
!kubectl get pods -n $NAMESPACE -l app=phi4

# 设置 Horizontal Pod Autoscaler (HPA)
!kubectl autoscale deployment phi4-deployment \
    --cpu-percent=70 \
    --min=1 \
    --max=3 \
    -n $NAMESPACE

# 查看 HPA 状态
!kubectl get hpa -n $NAMESPACE

## 清理资源

当您完成测试后，记得清理创建的资源以避免不必要的费用：

In [ ]:
# 删除部署
!kubectl delete -f k8s-manifests/

# 删除命名空间
!kubectl delete namespace $NAMESPACE

# 删除 ACR
!az acr delete \
    --name $ACR_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除 AKS 集群
!az aks delete \
    --name $AKS_CLUSTER_NAME \
    --resource-group $RESOURCE_GROUP \
    --yes

# 删除资源组（这将删除所有剩余资源）
!az group delete \
    --name $RESOURCE_GROUP \
    --yes

## 总结

本教程详细介绍了如何在 AKS 上使用 LLaMA-Factory Docker 方式部署微调后的模型：

### 主要步骤：
1. **环境准备**：创建 AKS 集群和 Azure Container Registry
2. **GPU 配置**：安装 NVIDIA GPU 设备插件
3. **镜像管理**：拉取并推送 LLaMA-Factory 镜像到 ACR
4. **Kubernetes 配置**：创建 ConfigMap、Deployment 和 Service 配置文件
5. **服务部署**：部署到 AKS 并配置负载均衡器
6. **测试验证**：通过 API 和 Web UI 测试模型功能
7. **监控扩展**：监控资源使用情况并配置自动扩展

### 优势：
- **云原生架构**：利用 Kubernetes 的编排和管理能力
- **高可用性**：支持多副本部署和负载均衡
- **弹性扩展**：自动扩展以应对负载变化
- **资源隔离**：通过命名空间和资源限制确保隔离
- **易于管理**：统一的部署和监控方式
- **成本效益**：不需要 Azure ML 的额外费用

### 注意事项：
- 确保 AKS 集群有足够的 GPU 资源
- 合理设置资源请求和限制
- 监控模型下载和启动时间
- 配置适当的健康检查探针
- 定期备份重要的配置文件
- 根据实际需求调整模型名称和适配器路径

### 自定义配置：
- 在 ConfigMap 中更新 `MODEL_NAME` 为您的实际模型名称
- 如果使用微调适配器，请设置 `ADAPTER_NAME` 
- 根据需要调整量化参数和资源限制
- 根据负载情况调整副本数量和 HPA 设置